In [3]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Our algorithms, by from the easiest to the hardest to intepret.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
df = pd.read_csv('C:/Users/Kaushik/Documents/Kaggle Datasets/Machine_Learning/Classification Datasets/Diabetes/Diabetes_all.csv')
print('Size of weather data frame is :',df.shape)

Size of weather data frame is : (768, 9)


In [4]:
df.describe()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df.count()

pregnant    768
glucose     768
bp          768
skin        768
insulin     768
bmi         768
pedigree    768
age         768
label       768
dtype: int64

In [6]:
def missing_values():
    temp_dict = dict()
    for i in df.columns:
        if df[i].isnull().sum()>0:
            temp_dict[i] = df[i].isnull().sum()
    return temp_dict   

In [7]:
missing_values()

{}

In [8]:
y = df["label"]
X = df.drop("label", axis=1)

In [9]:
dtypes_def = df.dtypes.reset_index()
dtypes_def.columns = ['Count','Column Type']
dtypes_def.groupby('Column Type').aggregate('count').reset_index()

,Column Type,Count
0,int64,7
1,float64,2


In [10]:
num_features = list(X.select_dtypes(include ='int64').columns)

In [11]:
num_features.extend(list(X.select_dtypes(include = 'float64').columns))

In [12]:
num_features

['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'age', 'bmi', 'pedigree']

In [13]:
preprocessor = ColumnTransformer([("numerical", "passthrough", num_features)])

In [14]:
# Logistic Regression
lr_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", LogisticRegression(class_weight="balanced", solver="liblinear", random_state=42))])

# Decision Tree
dt_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", DecisionTreeClassifier(class_weight="balanced"))])

# Random Forest
rf_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", RandomForestClassifier(class_weight="balanced", n_estimators=100, n_jobs=-1))])

# XGBoost
xgb_model = Pipeline([("preprocessor", preprocessor), 
                      # Add a scale_pos_weight to make it balanced
                      ("model", XGBClassifier(scale_pos_weight=(1 - y.mean()), n_jobs=-1))])

In [15]:
X_train,X_test, y_train, y_test = train_test_split(X,y,stratify = y, test_size = 0.3, random_state = 42)

In [16]:

from sklearn.metrics import accuracy_score
import time
t0=time.time()
xgb_model_1 = xgb_model
xgb_model.fit(X_train,y_train)
y_pred = xgb_model.predict(X_test)
score = accuracy_score(y_test,y_pred)
print('Accuracy :',score)
print('Time taken :' , time.time()-t0)

Accuracy : 0.7575757575757576
Time taken : 0.14883780479431152


In [17]:
# fine tune our logistic regression and evaluate its performance
gs = GridSearchCV(xgb_model, {"model__max_depth": [5, 10],
                              "model__min_child_weight": [5, 10],
                              "model__n_estimators": [25]}, n_jobs=-1, cv=5, scoring="accuracy")
gs.fit(X_train, y_train)    

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerical',
                                                                         'passthrough',
                                                                         ['pregnant',
                                                                          'glucose',
                                                                          'bp',
                                                                          'skin',
                                             

In [18]:
print(gs.best_params_)
print(gs.best_score_)

{'model__max_depth': 5, 'model__min_child_weight': 10, 'model__n_estimators': 25}
0.7615437867774316


In [19]:
xgb_model.set_params(**gs.best_params_)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['pregnant', 'glucose', 'bp',
                                                   'skin', 'insulin', 'age',
                                                   'bmi', 'pedigree'])],
                                   verbose=False)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=10, missing=None,
                               

In [20]:
xgb_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['pregnant', 'glucose', 'bp',
                                                   'skin', 'insulin', 'age',
                                                   'bmi', 'pedigree'])],
                                   verbose=False)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=5,
                               min_child_weight=10, missing=None,
                               

In [21]:
y_pred = xgb_model.predict(X_test)

In [22]:
accuracy_score(y_test, y_pred)

0.7532467532467533

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       150
           1       0.74      0.46      0.56        81

    accuracy                           0.75       231
   macro avg       0.75      0.69      0.70       231
weighted avg       0.75      0.75      0.74       231



In [24]:
# use eli5 to visualise the weights associated to each feature
import eli5

C:\Users\Kaushik\Anaconda3\envs\xai\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\Kaushik\Anaconda3\envs\xai\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [25]:
eli5.show_weights(xgb_model.named_steps["model"])

Weight,Feature
0.3540,f1
0.1457,f6
0.1421,f0
0.1261,f5
0.0632,f7
0.0630,f4
0.0602,f2
0.0456,f3


In [26]:
preprocessor = xgb_model.named_steps["preprocessor"]
model_1 = xgb_model.named_steps["model"]
all_features = num_features
pd.DataFrame(xgb_model.named_steps["preprocessor"].transform(X_train), columns=all_features).head()

,pregnant,glucose,bp,skin,insulin,age,bmi,pedigree
0,7.0,184.0,84.0,33.0,0.0,41.0,35.5,0.355
1,6.0,85.0,78.0,0.0,0.0,42.0,31.2,0.382
2,2.0,106.0,64.0,35.0,119.0,34.0,30.5,1.400
3,2.0,100.0,54.0,28.0,105.0,24.0,37.8,0.498
4,5.0,104.0,74.0,0.0,0.0,48.0,28.8,0.153


In [27]:
eli5.show_weights(xgb_model.named_steps["model"], feature_names=all_features)

Weight,Feature
0.3540,glucose
0.1457,bmi
0.1421,pregnant
0.1261,age
0.0632,pedigree
0.0630,insulin
0.0602,bp
0.0456,skin


In [28]:
i = 4
X_test.iloc[[i]]

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
387,8,105,100,36,0,43.3,0.239,45


In [29]:
eli5.show_prediction(xgb_model.named_steps["model"], 
                     xgb_model.named_steps["preprocessor"].transform(X_test)[i],
                     feature_names=all_features, show_feature_values=True)

In [30]:
# In order to explain why the model classifies invidividual observations as class 0 or 1, 
# we are going to use the LimeTabularExplainer from the library lime

from lime.lime_tabular import LimeTabularExplainer


In [31]:
explainer = LimeTabularExplainer(X_train.values,
                                 mode="classification",
                                 feature_names=X_train.columns.tolist(),
                                 discretize_continuous=True,
                                 random_state=42)

In [32]:
i = 2
X_observation = X_test.iloc[[i], :]
X_observation

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
24,11,143,94,33,146,36.6,0.254,51


In [33]:
observation = X_test.iloc[[i], :].values[0]
observation

array([ 11.   , 143.   ,  94.   ,  33.   , 146.   ,  36.6  ,   0.254,
        51.   ])

In [34]:
X_test.iloc[[i], :]

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age
24,11,143,94,33,146,36.6,0.254,51


In [37]:
# Let write a custom predict_proba functions for our models:
from functools import partial

def custom_predict_proba(X, model):
    X_str = X_train
    return model.predict_proba(X_str)

In [38]:
xgb_predict_proba = partial(custom_predict_proba, model=xgb_model)

In [ ]:
#explanation = explainer.explain_instance(observation, xgb_model.predict_proba, num_features=5)